In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import load_dataset

## Preprocessing and EDA

### a. Read dataset

In [32]:
path='C:/Users/user/Desktop/EPHI-Challenge/data/CHAMPS.csv'
df=load_dataset.load_data(path)

In [33]:
df.head()

,packet_version_id,id_ver_nmb,champs_id,dp_001,dp_002,dp_003,dp_004,dp_005,dp_006,dp_007,...,dpf_012___ch00040,dpf_012___ch00041,dpf_012___ch00042,dpf_012___ch00043,dpf_012___ch01424,dpf_012___ch01875,dpf_012___ch00010,dpf_013,dpf_014,crf_060302_decode_panel_feedback_form_complete
0,ETAA00002_01_01,2.0.0,ETAA00002,5,1,2,3,4.0,5.0,6.0,...,0,0,0,0,0,0,0,Tseyon Tesfaye Clinical,NaN,2
1,ETAA00004_01_02,2.0.0,ETAA00004,5,1,2,3,4.0,5.0,6.0,...,0,0,0,0,0,0,0,"Adugna (SBS team), Tigistu (counselor), Tseyon...",NaN,2
2,ETAA00005_01_02,2.0.0,ETAA00005,5,1,2,3,4.0,5.0,6.0,...,0,0,0,0,0,0,0,"Adugna (SBS team), Tigistu (counselor), Tseyon...",NaN,2
3,ETAA00008_01_04,2.0.0,ETAA00008,5,1,2,3,4.0,5.0,6.0,...,0,0,0,0,0,0,0,"Abdurahman Yonis(HO), Eyoel Taye(counselor), G...",The family leaves in a refugee camp and even t...,2
4,ETAA00009_01_01,2.0.0,ETAA00009,5,18,19,20,21.0,7.0,NaN,...,0,0,0,0,0,0,0,NaN,NaN,0


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444 entries, 0 to 443
Columns: 381 entries, packet_version_id to crf_060302_decode_panel_feedback_form_complete
dtypes: float64(106), int64(135), object(140)
memory usage: 1.3+ MB


### b. Number of columns and rows in dataset

In [35]:
df.shape

(444, 381)

### c. Enumerate the columns of the dataset

In [36]:
for index, column in enumerate(df.columns):
    print(f"{index}: {column}")

0: packet_version_id
1: id_ver_nmb
2: champs_id
3: dp_001
4: dp_002
5: dp_003
6: dp_004
7: dp_005
8: dp_006
9: dp_007
10: dp_008
11: dp_009
12: dp_010
13: dp_011
14: dp_012
15: dp_013
16: dp_153
17: dp_016
18: dp_017
19: dp_154
20: dp_155
21: dp_014
22: dp_108
23: dp_109
24: dp_110
25: dp_111
26: dp_112
27: dp_113
28: dp_114
29: dp_115
30: dp_116
31: dp_117
32: dp_157
33: dp_018
34: dp_019
35: dp_020
36: dp_021
37: dp_022
38: dp_023
39: dp_024
40: dp_025
41: dp_026
42: dp_027
43: dp_156
44: dp_028
45: dp_029
46: dp_030
47: dp_031
48: dp_032
49: dp_033
50: dp_034
51: dp_035
52: dp_036
53: dp_037
54: dp_038
55: dp_039
56: dp_040
57: dp_041
58: dp_042
59: dp_043
60: dp_044
61: dp_045
62: dp_046
63: dp_047
64: dp_048
65: dp_049
66: dp_050
67: dp_051
68: dp_052
69: dp_053
70: dp_054
71: dp_055
72: dp_056
73: dp_057
74: dp_058
75: dp_059
76: dp_060
77: dp_061
78: dp_062
79: dp_063
80: dp_064
81: dp_065
82: dp_066
83: dp_067
84: dp_068
85: dp_069
86: dp_070
87: dp_071
88: dp_072
89: dp_073
90

### d. Rename the columns.

In [37]:
df.rename(columns={'dp_013':'case_type'},inplace=True)

In [38]:
df['case_type']

0      CH00716
1      CH00716
2      CH00716
3      CH00719
4      CH01406
        ...   
439    CH00716
440    CH01404
441    CH00718
442    CH00719
443    CH01406
Name: case_type, Length: 444, dtype: object

In [39]:
df['case_type'] = df['case_type'].replace({'CH00716': 'Stillbirth', 
                                           'CH01404': 'Death in the first 24 hours',
                                           'CH01405':'Early Neonate',
                                           'CH01406':'Late Neonate',
                                           'CH00718':'Infant',
                                           'CH00719':'Child'})

### e. Rename values

In [40]:
df['case_type']

0                       Stillbirth
1                       Stillbirth
2                       Stillbirth
3                            Child
4                     Late Neonate
                  ...             
439                     Stillbirth
440    Death in the first 24 hours
441                         Infant
442                          Child
443                   Late Neonate
Name: case_type, Length: 444, dtype: object

In [41]:
# f. Show the proportion of null values in each column.
null_percentage =(df.apply(pd.isnull).sum()/df.shape[0])*100
for column, percentage in null_percentage.items():
    print(f"{column}: {percentage:.2f}%")

packet_version_id: 0.00%
id_ver_nmb: 0.00%
champs_id: 0.00%
dp_001: 0.00%
dp_002: 0.00%
dp_003: 0.00%
dp_004: 12.84%
dp_005: 16.89%
dp_006: 16.89%
dp_007: 18.47%
dp_008: 19.14%
dp_009: 26.35%
dp_010: 36.71%
dp_011: 56.08%
dp_012: 0.00%
case_type: 0.00%
dp_153: 46.17%
dp_016: 0.00%
dp_017: 93.69%
dp_154: 97.30%
dp_155: 6.31%
dp_014: 99.32%
dp_108: 0.00%
dp_109: 7.43%
dp_110: 93.02%
dp_111: 99.77%
dp_112: 98.20%
dp_113: 100.00%
dp_114: 98.87%
dp_115: 99.77%
dp_116: 94.59%
dp_117: 94.59%
dp_157: 20.05%
dp_018: 63.06%
dp_019: 63.51%
dp_020: 80.63%
dp_021: 99.55%
dp_022: 91.44%
dp_023: 99.55%
dp_024: 95.95%
dp_025: 100.00%
dp_026: 95.72%
dp_027: 95.72%
dp_156: 68.47%
dp_028: 79.50%
dp_029: 81.31%
dp_030: 87.61%
dp_031: 100.00%
dp_032: 95.27%
dp_033: 99.77%
dp_034: 98.65%
dp_035: 100.00%
dp_036: 97.75%
dp_037: 97.75%
dp_038: 87.84%
dp_039: 88.51%
dp_040: 93.92%
dp_041: 100.00%
dp_042: 97.30%
dp_043: 100.00%
dp_044: 99.32%
dp_045: 100.00%
dp_046: 98.65%
dp_047: 98.65%
dp_048: 93.69%
dp_049: 9

## Descriptive Data analysis

### a. The magnitude and proportion of each of the infant underlying cause for child death?

In [48]:
# Calculate magnitude
magnitude = df['dp_108'].value_counts()

# Calculate proportion
proportion = (magnitude / len(df)) * 100

In [49]:
magnitude

dp_108
Intrauterine hypoxia                                148
Birth asphyxia                                       33
Undetermined                                         28
Severe acute malnutrition                            24
Craniorachischisis                                   16
                                                   ... 
Severe acute malnutrition-Kwashiorkor                 1
severe acute malnutrition, Marasmic Kwashiorkor       1
Severe acute malnutrition - Marasmic kwashiorkor      1
Congenital CMV infection                              1
Bacterial sepsis of the newborn                       1
Name: count, Length: 97, dtype: int64

In [50]:
proportion

dp_108
Intrauterine hypoxia                                33.333333
Birth asphyxia                                       7.432432
Undetermined                                         6.306306
Severe acute malnutrition                            5.405405
Craniorachischisis                                   3.603604
                                                      ...    
Severe acute malnutrition-Kwashiorkor                0.225225
severe acute malnutrition, Marasmic Kwashiorkor      0.225225
Severe acute malnutrition - Marasmic kwashiorkor     0.225225
Congenital CMV infection                             0.225225
Bacterial sepsis of the newborn                      0.225225
Name: count, Length: 97, dtype: float64

### b. The proportion and magnitude of the maternal factors contributing for child death?

In [51]:
# Calculate magnitude
magnitude = df['dp_118'].value_counts()

# Calculate proportion
proportion = (magnitude / len(df)) * 100